# DataNights-VISceral
#### Código estructurado para las #DataNights de VISceral

<details>
    <summary><strong>Meta</strong></summary>
    La meta de este notebook es recrear el análisis de <a href="https://twitter.com/moaimx">@moaimx</a> que hizo en <a href="https://twitter.com/search?q=%23datanights&src=typed_query">#DataNights</a>
    <ul>
        <li> Limpiar las bases de datos del Secretariado sobre carpetas de investigación y víctimas de homicidios dolosos. </li>
        <ul>
            <li> Extraer y transformar el conjunto de datos <strong>Estatal-Delitos-2015-2019_mar19.zip</strong> y <strong>Estatal-Víctimas-2015-2019_mar19.zip</strong></li>
            <li> Guardar las bases de datos en formato .csv para facilitar el uso en un futuro. </li>
            <li> Crear una base de datos lista para visualización (en formato <i>tidy</i>). </li>
        </ul>
    </ul>
</details>

<details>
    <summary><strong>Contexto</strong></summary>
    Adquirimos los datos en bruto del <a href="http://www.morlan.mx/blog/2019/04/21/datanights-y-el-regreso-de-este-blog/">blog sobre #DataNights </a> en <a href="morlan.mx">morlan.mx</a>
    También seguiremos cercanamente los pasos para recrear lo que hicieron en los primeros dos episodios:
    <ul>
        <li><a href="https://www.youtube.com/watch?v=K2cYa41zmdw"> Episodio 1 </a></li>
        <li><a href="https://www.youtube.com/watch?v=xxQlCioKTe8"> Episodio 2 </a></li>
    </ul>
</details>

In [1]:
import pandas as pd
from zipfile import ZipFile
from pathlib import Path
from herramientas import arbol
from datetime import datetime as dt
hoy = dt.today().strftime("%d-%b-%y")

hoy

'30-May-19'

In [2]:
DATOS_BRUTOS = Path("../datos/brutos/")
DATOS_INTERINOS = Path("../datos/interinos/")
DATOS_PROCESADOS = Path("../datos/procesados/")
DATOS_EXTERNOS = Path("../datos/externos/")

In [3]:
arbol(DATOS_BRUTOS)

+ ..\datos\brutos
    + Estatal-Delitos-2015-2019_mar19.zip
    + Estatal-Víctimas-2015-2019_mar19.zip


***
## Extraer datos de un archivo .zip
Para extraer datos de un archivo `.zip` utilizamos la biblioteca 📚 de python `zipfile` de la siguiente manera:

<details>
    <summary>Nota</summary>
    En la primera celda de código de este notebook importamos el objeto <code style="background-color:#eeeeee">ZipFile</code> de la biblioteca <code style="background-color:#eee">zipfile</code>
</details>

In [4]:
archivo_zip_delitos = ZipFile(DATOS_BRUTOS / "Estatal-Delitos-2015-2019_mar19.zip")

<details>
    <summary>Nota sobre <code style="background-color:#eee">DATOS_BRUTOS / <span style="color:#ba2121">"Estatal-Delitos-2015-2019_mar19.zip"</span></code></summary>
    El objeto <code style="background-color:#eeeeee">Path</code> de la biblioteca <code style="background-color:#eee">pathlib</code> tiene la "habilidad" de conectar rutas de archivos utilizando el símbolo "/". Independientemente del sistema operativo que utilices. <br>
    Lo cual significa que si <code style="background-color:#eee">DATOS_BRUTOS / <span style="color:#ba2121">"Estatal-Delitos-2015-2019_mar19.zip"</span></code> es igual a <code style="background-color:#eee;color:#ba2121">"../datos/brutos/Estatal-Delitos-2015-2019_mar19.zip"</code> <br>
    o <code style="background-color:#eee;color:#ba2121">"..\datos\brutos\Estatal-Delitos-2015-2019_mar19.zip"</code> si estas en Windows (se formatea automaticamente).
</details>

Ahora `archivo_zip_delitos` es un objeto `ZipFile` así que podemos hacer lo siguiente:

In [6]:
# ver la lista de archivos que tiene
archivo_zip_delitos.filelist

[<ZipInfo filename='Estatal Delitos - marzo 2019.xlsb' compress_type=deflate external_attr=0x20 file_size=602500 compress_size=530072>]

In [7]:
# extraer los archivos

In [8]:
archivo_zip_delitos.extractall(path = DATOS_INTERINOS)

Ahora en nuestro archivo `datos/interinos/` existe el archivo `'Estatal Delitos - marzo 2019.xlsb'`

In [9]:
arbol(DATOS_INTERINOS)

+ ..\datos\interinos
    + Estatal Delitos - marzo 2019.xlsb


Hagamos lo mismo con el archivo de víctimas.

In [10]:
archivo_zip_victimas = ZipFile(DATOS_BRUTOS / 'Estatal-Víctimas-2015-2019_mar19.zip')
archivo_zip_victimas.extractall(DATOS_INTERINOS)

In [11]:
arbol(DATOS_INTERINOS)

+ ..\datos\interinos
    + Estatal Delitos - marzo 2019.xlsb
    + Estatal Víctimas - marzo 2019.xlsb


***
## Leer archivos Excel en `Pandas`

`Pandas` tiene la habilidad de leer archivos excel directamente pero un archivo excel esta compuesto de varias hojas así que leemos estas hojas en dos pasos:

In [12]:
archivo_delitos = pd.ExcelFile(DATOS_INTERINOS / 'Estatal Delitos - marzo 2019.xlsb', )

XLRDError: Excel 2007 xlsb file; not supported

Archivos XLSB (Excel Binario) no tienen soporte en las bibliotecas modernas de python pero existe una biblioteca no muy conocida llamada `pyxlsb` que nos puede ayudar. <br>
La puedes instalar con `pip` desde tu terminal:
```shell
pip install pyxlsb
```

In [13]:
import pyxlsb
# vamos a excribir csv a la antiüita
import csv

In [22]:
with pyxlsb.open_workbook(DATOS_INTERINOS / 'Estatal Delitos - marzo 2019.xlsb') as workbook:
    for nombre in workbook.sheets:
        with workbook.get_sheet(nombre) as hoja, open(DATOS_INTERINOS / f"delitos-{nombre}.csv", 'w',  encoding = 'utf-8') as f:
            escritor_csv = csv.writer(f)
            for fila in hoja.rows():
                escritor_csv.writerow([celda.v for celda in fila])

In [20]:
arbol(DATOS_INTERINOS)

+ ..\datos\interinos
    + delitos-Hoja1.csv
    + Estatal Delitos - marzo 2019.xlsb
    + Estatal Víctimas - marzo 2019.xlsb


Lo que estamos haciendo es:
1. Abrir el archivo y asignandolo a la variable "workbook"
2. Creamos una "loop" (o bucle) para hacer X ejercicio en cada hoja del workbook (`workbook.sheets`)
3. Creamos un archivo "delitos-{nombre de la hoja}.csv" 
4. Creamos un "escritor" de csv 
5. Hacemos otra "loop" (o bucle) y por cada fila en nuestra hoja creamos una lista con los valores de cada celda en esa fila la cual escribimos a nuestro archivo .csv <- esta es la parte más díficil de comprender porque son 3 pasos en uno.

Hagamos lo mismo con el archivo de víctimas:

In [23]:
with pyxlsb.open_workbook(DATOS_INTERINOS / 'Estatal Víctimas - marzo 2019.xlsb') as workbook:
    for nombre in workbook.sheets:
        with workbook.get_sheet(nombre) as hoja, open(DATOS_INTERINOS / f"victimas-{nombre}.csv", 'w', encoding = 'utf-8') as f:
            escritor_csv = csv.writer(f)
            for fila in hoja.rows():
                escritor_csv.writerow([celda.v for celda in fila])

In [25]:
arbol(DATOS_INTERINOS)

+ ..\datos\interinos
    + delitos-Hoja1.csv
    + Estatal Delitos - marzo 2019.xlsb
    + Estatal Víctimas - marzo 2019.xlsb
    + victimas-Hoja1.csv


Esta "loop" crea un archivo `.csv` por cada hoja en el archivo `.xlsb` así que este pedacito de código funciona aunque tu archivo excel tenga 150 hojas. 

***
## Creando un conjunto de datos listos para visualización (en formato `tidy`)

Hasta el momento llevamos 2 de nuestras 3 metas:<br>
<input type="checkbox" checked> Extraer y transformar el conjunto de datos <strong>Estatal-Delitos-2015-2019_mar19.zip</strong> y <strong>Estatal-Víctimas-2015-2019_mar19.zip</strong><br>
<input type="checkbox" checked> Guardar las bases de datos en formato .csv para facilitar el uso en un futuro.<br>
<input type="checkbox" > Crear una base de datos lista para visualización (en formato <i>tidy</i>).

El limpiar y restructurar datos siempre es lo que lleva más tiempo en el proceso de análisis de datos.

In [26]:
delitos = pd.read_csv(DATOS_INTERINOS / 'delitos-Hoja1.csv')
victimas = pd.read_csv(DATOS_INTERINOS / 'victimas-Hoja1.csv')

In [27]:
delitos.head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0
1,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0.0,0.0,2.0,2.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
3,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
victimas.head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Sexo,Rango de edad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NOTA:
Este tutorial **asume** que viste los episodios 1 y 2 de #DataNights. No vamos a explicar porque estamos escogiendo las variables que estamos escogiendo o lo que ilustran cada uno de estos conjuntos de datos.

El producto final de ambos conjuntos (víctimas y delitos) es parecido: un conjunto de datos con las columnas "Año", "Mes", "Delitos" o "Víctimas" donde esta la cuenta total (agregada) de los delitos o víctimas en México en ese Año-Mes.

### Filtrando datos
Primero vemos los valores _unicos_ en nuestra columna 'Supbipo de delito' para asegurarnos de que no haya errores de dedo. Por ejemplo, en bases de datos de baja calidad puedes encontrar:
1. "Homicidio doloso"
2. "Homicidio doloso "
3. " Homicidio doloso"
4. "homicidio doloso"
5. "Homicidio Doloso"

Todos estos valores son distintos para `pandas`. 

In [31]:
delitos['Subtipo de delito'].unique()

array(['Homicidio doloso', 'Homicidio culposo', 'Lesiones dolosas',
       'Lesiones culposas', 'Feminicidio', 'Aborto',
       'Otros delitos que atentan contra la vida y la integridad corporal',
       'Secuestro', 'Tráfico de menores', 'Rapto',
       'Otros delitos que atentan contra la libertad personal',
       'Abuso sexual', 'Acoso sexual', 'Hostigamiento sexual',
       'Violación simple', 'Violación equiparada', 'Incesto',
       'Otros delitos que atentan contra la libertad y la seguridad sexual',
       'Robo a casa habitación', 'Robo de vehículo automotor',
       'Robo de autopartes', 'Robo a transportista',
       'Robo a transeúnte en vía pública',
       'Robo a transeúnte en espacio abierto al público',
       'Robo en transporte público individual',
       'Robo en transporte público colectivo',
       'Robo en transporte individual', 'Robo a institución bancaria',
       'Robo a negocio', 'Robo de ganado', 'Robo de maquinaria',
       'Otros robos', 'Fraude', 'Abuso

Ya que vemos que nuestra columna 'Subtipo de delito' si está bien construida/limpia podemos crear "máscaras" para filtrar los datos.

In [35]:
mascara_homicidio_doloso_delitos = delitos['Subtipo de delito'] == 'Homicidio doloso'
mascara_homicidio_doloso_victimas = victimas['Subtipo de delito'] == 'Homicidio doloso'

Para más información sobre como funciona esto ve los videos de DataNights.

In [36]:
delitos[mascara_homicidio_doloso_delitos].head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0
1,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0.0,0.0,2.0,2.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
3,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2015.0,2.0,Baja California,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,20.0,26.0,33.0,42.0,49.0,43.0,46.0,50.0,57.0,44.0,51.0,36.0


In [37]:
victimas[mascara_homicidio_doloso_victimas].head()

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Sexo,Rango de edad,Enero,...,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,Mujer,Menores de edad (0-17),0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,2015.0,1.0,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,Mujer,Adultos (18 y más),0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Nota el _brinco_ en el índice en ambos **DataFrames**. Esto significa que hemos eliminado todas las filas que no cumplan el requisito de la "máscara" (que la columna "Subtipo de delito" equivalga "Homicidio doloso").

In [38]:
delitos = delitos[mascara_homicidio_doloso_delitos].copy()
victimas = victimas[mascara_homicidio_doloso_victimas].copy()

<details>
    <summary>Nota sobre <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code></summary>
    Utilizar <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code> no es necesario ya que estamos sobreescribiendo la variable delitos. <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code> es necesario cuando estas creando una copia de una parte de otro DataFrame. Pandas por defecto va a crear una especie de "vista" en lugar de una copia copia. Esto puede tener consecuencias no deseadas más adelante en tus análisis. Así que es buena practica utilizar <code style="background-color:#eee">.<span style="color:#0055AA">copy</span>()</code>
</details>

### Trabajando con fechas
`python` es muy bueno trabajando con fechas. En la biblioteca estándard (que viene incluida en `python`) existe el paquete `datetime` para trabajar con fechas y tiempos. Es posible utilizar este paquete para trabajar con fechas en español también (sólo es cuestión de cambiar el `LC_TIME`) pero existen otros paquetes como `arrow` que hacen el proceso de trabajar con fechas mucho más amigable.

In [41]:
import arrow

In [43]:
arrow.locales.SpanishLocale.month_names

['',
 'enero',
 'febrero',
 'marzo',
 'abril',
 'mayo',
 'junio',
 'julio',
 'agosto',
 'septiembre',
 'octubre',
 'noviembre',
 'diciembre']

`Arrow` tiene "locales" para muchos muchos idiomas. En este caso vamos a utilizar el `SpanishLocale` para obtener una lista de los meses más rapidamente que escribiendolos a mano.

In [45]:
# esta es una list comprehension - ve el video de DataNights para aprender más sobre esto
meses = [mes.title() for mes in arrow.locales.SpanishLocale.month_names if mes]
meses

['Enero',
 'Febrero',
 'Marzo',
 'Abril',
 'Mayo',
 'Junio',
 'Julio',
 'Agosto',
 'Septiembre',
 'Octubre',
 'Noviembre',
 'Diciembre']

Nota: agregamos el `if mes` para decirle a `python` que me de un `mes` con la primera letra en mayúscula (`.title()`) **sólo** si `mes` existe/es `True`. En `python` las cadenas ( o _strings_) tienen un valor equivalente a `False` si están vacías (como el primer valor de la lista `arrow.locales.SpanishLocale.month_names`).

Ahora podemos crear una lista de "columnas de interés", las columnas que vamos a utilizar en nuestro análisis. Aquí es de donde partimos un poco del proceso utilizado en DataNights. 

Hacemos esto por preferencia personal. Antes de restructurar conjuntos de datos, a mí, en lo personal, me gusta filtrar todo lo que no voy a utilizar. Esto significa, eliminar filas **y** columnas que no crea necesarias para mi análisis. 

Primero creamos una lista de columnas que nos interesan. En este caso van a ser el Año y los meses. Como ya tenemos una lista de meses lo que hacemos es `extend`er una primera lista (que solo tiene "Año") para incluir los meses.

In [46]:
columnas_de_interes = ['Año']
columnas_de_interes.extend(meses)
columnas_de_interes

['Año',
 'Enero',
 'Febrero',
 'Marzo',
 'Abril',
 'Mayo',
 'Junio',
 'Julio',
 'Agosto',
 'Septiembre',
 'Octubre',
 'Noviembre',
 'Diciembre']

Esto funciona igual que las máscaras que utilizamos anteriormente para filtrar filas. Y ya que ambos **DataFrame**s tienen las mismas columnas, no ocupamos crear dos "máscaras".

In [47]:
delitos[columnas_de_interes].head()

,Año,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,3.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0
1,2015.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2015.0,0.0,0.0,2.0,2.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
3,2015.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2015.0,20.0,26.0,33.0,42.0,49.0,43.0,46.0,50.0,57.0,44.0,51.0,36.0


In [48]:
victimas[columnas_de_interes].head()

,Año,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,2015.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [49]:
delitos = delitos[columnas_de_interes].copy()
victimas = victimas[columnas_de_interes].copy()

### Derretir nuestos datasets
Reitero: No vamos a explicar todo esto en este tutorial. Si no sabes lo que estamos haciendo aquí, ve a ver los episodios 1 y 2 de #DataNights.

In [68]:
delitos_derretido = delitos.melt(id_vars = 'Año', var_name = 'Mes', value_name = 'Cuenta (carpetas)')
victimas_derretido = victimas.melt(id_vars = 'Año', var_name = 'Mes', value_name = 'Cuenta (victimas)')

Ya que tenemos nuestros nuevos **DataFrame**s en esta nueva estructura (derretida) es cuando yo modificaría los valores.

In [53]:
victimas_derretido.head()

,Año,Mes,Cuenta (victimas)
0,2015.0,Enero,0.0
1,2015.0,Enero,0.0
2,2015.0,Enero,0.0
3,2015.0,Enero,0.0
4,2015.0,Enero,0.0


Necesitamos un _diccionario_ con los nombres del mes como llave y sus valores correspondientes (01-12). 

Igual que hicimos una "list comprehension" podemos hacer un "dict comprehension"

In [55]:
{mes: valor for valor, mes in enumerate(meses, start = 1)}

{'Enero': 1,
 'Febrero': 2,
 'Marzo': 3,
 'Abril': 4,
 'Mayo': 5,
 'Junio': 6,
 'Julio': 7,
 'Agosto': 8,
 'Septiembre': 9,
 'Octubre': 10,
 'Noviembre': 11,
 'Diciembre': 12}

Pero queremos los valores en formato "01" en lugar de 1. Para eso utilizamos `.zfill()`

In [56]:
{mes: str(valor).zfill(2) for valor, mes in enumerate(meses, start = 1)}

{'Enero': '01',
 'Febrero': '02',
 'Marzo': '03',
 'Abril': '04',
 'Mayo': '05',
 'Junio': '06',
 'Julio': '07',
 'Agosto': '08',
 'Septiembre': '09',
 'Octubre': '10',
 'Noviembre': '11',
 'Diciembre': '12'}

In [57]:
sustituciones_meses = {mes: str(valor).zfill(2) for valor, mes in enumerate(meses, start = 1)}

Ahora podemos sustituir nuestra columna de "Mes" por valores númericos (más o menos númericos).

In [69]:
victimas_derretido["Mes"] = victimas_derretido["Mes"].map(sustituciones_meses)
delitos_derretido["Mes"] = delitos_derretido["Mes"].map(sustituciones_meses)

In [70]:
victimas_derretido.head()

,Año,Mes,Cuenta (victimas)
0,2015.0,01,0.0
1,2015.0,01,0.0
2,2015.0,01,0.0
3,2015.0,01,0.0
4,2015.0,01,0.0


Ahora transformamos nuestra columna "Año" a _string_ para concatenarla con "Mes" para tener una columna "Año-Mes" que utilizaremos como nuestro eje X.

Primero la convertimos a `int` o enteros para deshacernos del .0 al final de cada número.

In [71]:
victimas_derretido['Año'] = victimas_derretido['Año'].astype(int).astype(str)
delitos_derretido['Año'] = delitos_derretido['Año'].astype(int).astype(str)

In [72]:
victimas_derretido.head()

,Año,Mes,Cuenta (victimas)
0,2015,01,0.0
1,2015,01,0.0
2,2015,01,0.0
3,2015,01,0.0
4,2015,01,0.0


In [73]:
victimas_derretido['Fecha'] = victimas_derretido["Año"] + '-' + victimas_derretido['Mes']
delitos_derretido['Fecha'] = delitos_derretido["Año"] + '-' + delitos_derretido['Mes']

In [74]:
victimas_derretido.head()

,Año,Mes,Cuenta (victimas),Fecha
0,2015,01,0.0,2015-01
1,2015,01,0.0,2015-01
2,2015,01,0.0,2015-01
3,2015,01,0.0,2015-01
4,2015,01,0.0,2015-01


Ya que tenemos estos valores "Fecha" podemos agrupar nuestros datos por esta columna y agregarlos:

In [77]:
victimas_derretido.groupby("Fecha")['Cuenta (victimas)'].sum().head()

Fecha
2015-01    1354.0
2015-02    1388.0
2015-03    1379.0
2015-04    1442.0
2015-05    1601.0
Name: Cuenta (victimas), dtype: float64

In [78]:
victimas_derretido.groupby("Fecha")[['Cuenta (victimas)']].sum().head()

,Cuenta (victimas)
Fecha,
2015-01,1354.0
2015-02,1388.0
2015-03,1379.0
2015-04,1442.0
2015-05,1601.0


Nota que si utilizas [["___________"]] en lugar de un solo par de corchetes recibirás un **DataFrame** en lugar de una **Serie** de pandas por defecto.

In [80]:
victimas_agrupado = victimas_derretido.groupby("Fecha")[['Cuenta (victimas)']].sum().reset_index()
delitos_agrupado = delitos_derretido.groupby("Fecha")[['Cuenta (carpetas)']].sum().reset_index()

In [83]:
victimas_agrupado.head()

,Fecha,Cuenta (victimas)
0,2015-01,1354.0
1,2015-02,1388.0
2,2015-03,1379.0
3,2015-04,1442.0
4,2015-05,1601.0


Pero como vimos en #DataNights, nuestro conjunto de datos tiene columnas para todos los meses del año pero valores solo hasta marzo 2019. Esto significa que empezando en abril, los valores son en nuestro conjunto modificado son 0:

In [84]:
victimas_agrupado.tail()

,Fecha,Cuenta (victimas)
55,2019-08,0.0
56,2019-09,0.0
57,2019-10,0.0
58,2019-11,0.0
59,2019-12,0.0


Como nuestros datos estan ordenados podemos hacer algo como

In [90]:
victimas_agrupado[:-9]

,Fecha,Cuenta (victimas)
0,2015-01,1354.0
1,2015-02,1388.0
2,2015-03,1379.0
3,2015-04,1442.0
4,2015-05,1601.0
5,2015-06,1467.0
6,2015-07,1539.0
7,2015-08,1675.0
8,2015-09,1502.0
9,2015-10,1464.0


eliminando las ultimas 9 filas del **DataFrame**. También sabemos ningún par de Año-Mes tiene un valor equivalente a 0 de manera natural así que podemos crear una máscara para filtrar las filas que no cumplan este requisito. 

In [91]:
mascara_cuenta_victimas = victimas_agrupado['Cuenta (victimas)'] > 0
mascara_cuenta_delitos = delitos_agrupado['Cuenta (carpetas)'] > 0

In [92]:
delitos_agrupado[mascara_cuenta_delitos]

,Fecha,Cuenta (carpetas)
0,2015-01,1197.0
1,2015-02,1218.0
2,2015-03,1235.0
3,2015-04,1297.0
4,2015-05,1410.0
5,2015-06,1361.0
6,2015-07,1360.0
7,2015-08,1517.0
8,2015-09,1382.0
9,2015-10,1352.0


In [93]:
delitos_agrupado = delitos_agrupado[mascara_cuenta_delitos].copy()
victimas_agrupado = victimas_agrupado[mascara_cuenta_victimas].copy()

# ¡Listo!
Tenemos nuestros conjuntos de datos listos para visualizar. 
Guardemoslos en formato `.csv` para facilitar su uso en un futuro.

In [94]:
delitos_agrupado.to_csv(DATOS_PROCESADOS / f'carpetas-cuenta-{hoy}.csv', encoding = 'utf-8', index = False)
victimas_agrupado.to_csv(DATOS_PROCESADOS / f'victimas-cuenta-{hoy}.csv', encoding = 'utf-8', index = False)

***
## Visualización
Para mantener nuestro entorno de trabajo organizado crearemos nuestra visualización en otro notebook. 